### Cross encoder 구조

- 이 글은 Cross Encoder의 구조를 소개하고 학습하는 방법을 설명함. 

- Cross encoder는 Pretrained Model에 Classification layer를 쌓은 구조임. 

- Cross Encoder는 🤗Transforemers의 Sequenceclassification model을 불러와 쉽게 구현할 수 있음.

- Sentence_transformers 라이브러리에서 구현된 Cross_Encoder 또한 내부에 SequenceClassification를 기반으로 구성됨.


### model 불러오기

* Cross Encoder는 Pretrained Model에 Classification Head를 연결한 모델임.

    <img src='../images/cross_encoder.png' alt='cross_encoder' width='500px'>

#### ClassificationHead

- Classification 상세 구조는 dense_layer => gelu => output_pojection_layer로 되어있음.

    <img src='../images/classification_head.png' alt='classification_head' width='500px'>


<br/>

- Pre-trained Model의 output인 last_hidden_state 중 [CLS] 토큰만을 Classification Head의 input data로 활용

- 수많은 토큰 embedding이 있지만 그 중 [CLS] 토큰이 두 문장의 관계를 요약한 embedding이라 판단하므로 이를 활용함.  

- 이처럼 여러 정보를 하나로 치환하는 방법을 pooling이라 함. pooling 관련해서는 [Why it called pooler?](https://github.com/google-research/bert/issues/1102)를 참고.

- pooling 된 embedding은 ReLU(또는 Tanh) 함수를 거친 다음 projection layer를 통해 라벨 크기에 맞는 차원으로 축소함.

- Regression 데이터로 학습하는 경우 라벨 개수(N)은 1로 설정해야하며, Classification 데이터로 학습하는 경우 라벨 개수에 맞게 N을 설정해야함.


In [ ]:
from transformers import ElectraModel, ElectraTokenizerFast, TrainingArguments, Trainer
from torch.nn import CrossEntropyLoss, MSELoss
from datasets import Dataset
from torch import Tensor, nn
import pandas as pd
import torch

class classificationHead(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        classifier_dropout = (
            config.classifier_dropout
            if config.classifier_dropout is not None
            else config.hidden_dropout_prob
        )
        self.gelu = nn.functional.gelu

        self.dropout = nn.Dropout(classifier_dropout)

        # [batch, embed_size] => [batch, num_labels]
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features[:, 0, :]  # [CLS] 토큰 추출
        x = self.dropout(x)
        x = self.dense(x)
        x = self.gelu(x)
        x = self.dropout(x)

        # label 개수만큼 차원 축소 [batch, embed_size] => [batch, num_labels]
        x = self.out_proj(x)

        return x


### CrossEncoder 구조

- 아래의 CrossEncoder 구조는 🤗Transforemers의 Sequenceclassification 내부 코드를 기반으로 작성했으며, 이해하기 쉽도록 코드 일부를 변경하였음.

- Cross_Encoder의 Output은 모델 학습 시 Loss와 Logits을 반환하고 평가 및 활용 시에는 Logits만 반환함.

- 학습에 데이터에 따라 Loss_function이 달라짐. 학습 유형이 Regression일 때 MSE, Classfication일 때 Cross-Enctropy를 활용함.

- 학습 유형에 따라 Loss Function이 달라지는 이유가 궁금한 경우 다음을 참고
    - [In which cases is the cross-entropy preferred over the mean squared error?](https://stackoverflow.com/questions/36515202/in-which-cases-is-the-cross-entropy-preferred-over-the-mean-squared-error)
    
    - [What is the different between MSE error and Cross-entropy error in NN](https://susanqq.github.io/tmp_post/2017-09-05-crossentropyvsmes/)



In [ ]:
class CrossEncoder(nn.Module):
    def __init__(self, model, num_labels) -> None:
        super().__init__()
        self.model = model
        self.model.config.num_labels = num_labels
        self.classifier = classificationHead(self.model.config)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):

        discriminator_hidden_states = self.model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # Last-hidden-state 추출
        sequence_output = discriminator_hidden_states[0]

        # Last-hidden-state를 classificationHead의 입력 데이터로 활용
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            if self.model.config.num_labels == 1:
                # Regression Model은 MSE Loss 활용
                loss_fct = MSELoss()
            else:
                # classification Model은 Cross entropy 활용
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, 3), labels.view(-1))
            return {"loss": loss, "logit": logits}
        else:
            return {"logit": logits}


### Cross Encoder 학습하기
* 이 글에선 Numerical Data를 활용해 Cross Encoder를 학습하는 방법만을 소개함.

### KorSTS Data 불러오기


In [ ]:
with open("../data/KorSTS/sts-train.tsv") as f:
    v = f.readlines()

## from list to dataframe
lst = [i.rstrip("\n").split("\t") for i in v]

data = pd.DataFrame(lst[1:], columns=lst[:1])
data = data[["sentence1", "sentence2", "score"]]
data.columns = ["sen1", "sen2", "score"]
data.head(3)


,sen1,sen2,score
0,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.,5.000
1,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.,3.800
2,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.,3.800


### Huggingface Datasets으로 변환

- 🤗Transformers와 호환을 위해 Dataframe을 🤗dataset으로 변환


In [ ]:
train_data_set = Dataset.from_pandas(data)

train_data_set[0]


{'sen1': '비행기가 이륙하고 있다.', 'sen2': '비행기가 이륙하고 있다.', 'score': '5.000'}

### collator 구현

- 학습 구조에 맞는 input data 생성을 위한 custom collator 제작


In [1]:
def smart_batching_collate(batch):
    text_lst1 = []
    text_lst2 = []
    labels = []

    for example in batch:
        for k, v in example.items():
            if k == "sen1":
                text_lst1.append(v)
            if k == "sen2":
                text_lst2.append(v)
            if k == "score":
                labels.append(float(v))

    labels = torch.tensor(labels)

    sentence_features = []
    for items in [text_lst1, text_lst2]:
        token = tokenizer(items, return_tensors="pt", truncation=True, padding=True)
        sentence_features.append(token)

    return dict(features=sentence_features, answer=labels)


### 🤗Transformers Trainer를 활용해 학습


In [ ]:
# Trainer Option 설정

training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=10,
    eval_steps=10,
    num_train_epochs=2,
    remove_unused_columns=False,
    evaluation_strategy="steps",
    save_steps=2000,
)

# 모델 불러오기
model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer = ElectraTokenizerFast.from_pretrained("monologg/koelectra-base-v3-discriminator")
cross_encoder = CrossEncoder(model, num_labels=1) 

# Huggingface의 ElectraForSequenceClassification을 cross_encoder로 활용가능
# from transformers import ElectraForSequenceClassification
# cross_encoder = ElectraForSequenceClassification.from_pretrained('model/disc_book_final',num_labels=3)

# Trainer 정의
trainer = Trainer(
    model=cross_encoder,
    train_dataset=train_data_set,
    args=training_args,
    data_collator=smart_batching_collate,
)

trainer.train()
